In [1]:
import pandas as pd
import numpy as np

from surprise import accuracy
from surprise import SVD, NMF
from surprise import Dataset
from surprise import Reader
from surprise import NormalPredictor
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV

In [3]:
# Read in the ratings_small (user-item) dataframe
df_ratings = pd.read_csv('data/ratings.csv')

In [4]:
df_all = pd.read_csv('data/dataframe_merged.csv', usecols=['id', 'title'])

In [5]:
df_ratings = df_ratings[df_ratings['movieId'].isin(df_all['id'])]

In [6]:
df_ratings.tail(10)

,userId,movieId,rating,timestamp
999979,10183,348,3.0,833460122
999980,10183,349,5.0,833459462
999981,10183,350,4.0,833460139
999987,10183,364,5.0,833459740
999993,10183,377,5.0,833460368
999994,10183,378,5.0,833461211
999995,10183,380,4.0,833459432
999996,10183,381,4.0,833460156
999997,10183,410,3.0,833459559
999998,10183,415,4.0,833460564


In [7]:
# Set the reader to have a rating_scale from 1-5 (default)
reader = Reader(rating_scale=(1, 5))

In [8]:
# The data only consists of userId, movieId, and rating
data = Dataset.load_from_df(df_ratings[['userId', 'movieId', 'rating']], reader)

In [9]:
# Use surprise package for a train-test split of 80-20
# Note that the train-test split will split by general rows, not specific users
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.20)

In [10]:
# trainsetfull = data.build_full_trainset()

In [11]:
print('Number of users: ', trainset.n_users)
print('Number of movies (items): ', trainset.n_items, '\n')

Number of users:  9905
Number of movies (items):  4818 



In [12]:
trainset_iids = list(trainset.all_items())
iid_converter = lambda x: trainset.to_raw_iid(x)
trainset_raw_iids = list(map(iid_converter, trainset_iids))

In [13]:
trainset_raw_iids

[58559,
 4638,
 494,
 527,
 25,
 2928,
 6078,
 1393,
 68,
 8951,
 2770,
 260,
 4643,
 2019,
 223,
 2671,
 780,
 3527,
 1954,
 1977,
 5995,
 2160,
 8983,
 344,
 1682,
 1584,
 608,
 924,
 6461,
 3525,
 594,
 1676,
 3081,
 164,
 440,
 1213,
 842,
 539,
 40732,
 902,
 1937,
 95,
 1580,
 2333,
 1259,
 920,
 30793,
 1892,
 1884,
 916,
 616,
 2302,
 1968,
 2890,
 4226,
 1059,
 750,
 805,
 745,
 3210,
 5,
 296,
 71184,
 434,
 6947,
 314,
 500,
 597,
 54503,
 1373,
 134368,
 2349,
 543,
 910,
 648,
 44974,
 1573,
 1722,
 8340,
 714,
 377,
 4975,
 1380,
 1391,
 1680,
 2255,
 586,
 524,
 2288,
 225,
 185,
 6373,
 364,
 177,
 167738,
 535,
 2125,
 2617,
 88,
 837,
 922,
 1641,
 4326,
 110,
 2161,
 2001,
 318,
 2124,
 480,
 2501,
 1648,
 3683,
 3175,
 111,
 1963,
 593,
 36529,
 74154,
 2291,
 4912,
 94959,
 33166,
 801,
 2912,
 923,
 4701,
 2109,
 107406,
 380,
 193,
 1396,
 1249,
 196,
 84944,
 942,
 553,
 1610,
 2021,
 2117,
 4993,
 6,
 3028,
 4025,
 595,
 19,
 31696,
 1266,
 1265,
 350,
 347,
 3

In [14]:
from surprise import KNNWithMeans
my_k = 15
my_min_k = 5
my_sim_option = {
    'name':'cosine', 'user_based':False, 'verbose': False
    }
algo = KNNWithMeans(sim_options = my_sim_option)
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [15]:
# Dataframe of all the movies voted for
pd.DataFrame(algo.sim)

,0,1,2,3,4,5,6,7,8,9,...,4808,4809,4810,4811,4812,4813,4814,4815,4816,4817
0,1.000000,0.935286,0.973504,0.958661,0.960105,1.000000,0.944771,0.952749,1.000000,0.961962,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
1,0.935286,1.000000,0.942032,0.931058,0.945446,1.000000,0.952438,0.935474,0.000000,0.995950,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.973504,0.942032,1.000000,0.952673,0.920150,1.000000,0.954213,0.946754,0.939411,0.966753,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.958661,0.931058,0.952673,1.000000,0.956590,0.964376,0.895596,0.967638,0.938939,0.978285,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.960105,0.945446,0.920150,0.956590,1.000000,0.882850,0.952228,0.943133,0.983317,0.963145,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4813,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4814,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4815,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4816,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [16]:
# Same dataframe as algo.sim but the indices/columns are now movieId
df_cos_surprise = pd.DataFrame(algo.sim, index=trainset_raw_iids, columns=trainset_raw_iids)

df_cos_surprise

,58559,4638,494,527,25,2928,6078,1393,68,8951,...,8453,75438,136345,46525,133131,85312,87302,114426,61638,57276
58559,1.000000,0.935286,0.973504,0.958661,0.960105,1.000000,0.944771,0.952749,1.000000,0.961962,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
4638,0.935286,1.000000,0.942032,0.931058,0.945446,1.000000,0.952438,0.935474,0.000000,0.995950,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
494,0.973504,0.942032,1.000000,0.952673,0.920150,1.000000,0.954213,0.946754,0.939411,0.966753,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
527,0.958661,0.931058,0.952673,1.000000,0.956590,0.964376,0.895596,0.967638,0.938939,0.978285,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
25,0.960105,0.945446,0.920150,0.956590,1.000000,0.882850,0.952228,0.943133,0.983317,0.963145,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85312,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
87302,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
114426,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
61638,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [17]:
df_all

,id,title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II
...,...,...
46623,439050,Subdue
46624,111109,Century of Birthing
46625,67758,Betrayal
46626,227506,Satan Triumphant


In [18]:
df_all = df_all.reset_index()
df_all.index = df_all.id

In [19]:
df_all

,index,id,title
id,,,
862,0,862,Toy Story
8844,1,8844,Jumanji
15602,2,15602,Grumpier Old Men
31357,3,31357,Waiting to Exhale
11862,4,11862,Father of the Bride Part II
...,...,...,...
439050,46623,439050,Subdue
111109,46624,111109,Century of Birthing
67758,46625,67758,Betrayal


In [20]:
movieIdtoindex = df_all['index'].to_dict()

In [21]:
df_cos_surprise = df_cos_surprise.rename(index=movieIdtoindex, columns=movieIdtoindex)

In [22]:
df_cos_surprise

,38888,11726,3686,286,10612,1379,19853,12101,1178,16680,...,45745,46013,29512,21725,45980,22232,22102,39446,31965,17341
38888,1.000000,0.935286,0.973504,0.958661,0.960105,1.000000,0.944771,0.952749,1.000000,0.961962,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
11726,0.935286,1.000000,0.942032,0.931058,0.945446,1.000000,0.952438,0.935474,0.000000,0.995950,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3686,0.973504,0.942032,1.000000,0.952673,0.920150,1.000000,0.954213,0.946754,0.939411,0.966753,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
286,0.958661,0.931058,0.952673,1.000000,0.956590,0.964376,0.895596,0.967638,0.938939,0.978285,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
10612,0.960105,0.945446,0.920150,0.956590,1.000000,0.882850,0.952228,0.943133,0.983317,0.963145,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22232,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
22102,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
39446,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
31965,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [23]:
# Make a pandas dataframe of movie x movie length from df_all
# Fill in the values from matrix 'algo.sim'
# Set the diagonal to "1"
df_blank = pd.DataFrame(np.nan, range(1,len(df_all)), range(1,len(df_all)))
df_blank

,1,2,3,4,5,6,7,8,9,10,...,46618,46619,46620,46621,46622,46623,46624,46625,46626,46627
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46624,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_blank = df_cos_surprise.combine_first(df_blank)

In [ ]:
df_blank

In [ ]:
# # Assign the diagonals as 1
# df_blank.values[[np.arange(df_blank.shape[0])]*2] = 1

In [ ]:
np.fill_diagonal(df_blank.values, 1)
df_blank

In [ ]:
df_bank = df_blank.fillna(0)

In [ ]:
df_blank

In [ ]:
np.save('cosine_similarity/cos_ratings.npy', df_blank)